In [1]:
import tensorflow as tf
import numpy as np
import pickle

tf.compat.v1.disable_eager_execution()
# Create model
def create_multilayer_perceptron():
    # Network Parameters
    n_hidden_1 = 256  # 1st layer number of features
    n_hidden_2 = 256  # 2nd layer number of features
    n_input = 2376    # data input
    n_classes = 2     # number of classes

    # Store layers weight & bias
    weights = {
        'h1': tf.compat.v1.Variable(tf.random.normal([n_input, n_hidden_1])),
        'h2': tf.compat.v1.Variable(tf.random.normal([n_hidden_1, n_hidden_2])),
        'out': tf.compat.v1.Variable(tf.random.normal([n_hidden_2, n_classes]))
    }
    biases = {
        'b1': tf.compat.v1.Variable(tf.random.normal([n_hidden_1])),
        'b2': tf.compat.v1.Variable(tf.random.normal([n_hidden_2])),
        'out': tf.compat.v1.Variable(tf.random.normal([n_classes]))
    }
    
    # tf Graph input
    x = tf.compat.v1.placeholder("float", [None, n_input])
    y = tf.compat.v1.placeholder("float", [None, n_classes])

    # Hidden layer with ReLU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)

    # Hidden layer with ReLU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)

    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer, x, y


In [2]:
# Data preprocessing function
def preprocess():
    with open('basecode/face_all.pickle', 'rb') as file:
        pickle_obj = pickle.load(file)
    features = pickle_obj['Features']
    labels = pickle_obj['Labels']
    train_x = features[0:21100] / 255
    valid_x = features[21100:23765] / 255
    test_x = features[23765:] / 255

    labels = labels.T
    train_y = np.zeros(shape=(21100, 2))
    train_l = labels[0:21100]
    valid_y = np.zeros(shape=(2665, 2))
    valid_l = labels[21100:23765]
    test_y = np.zeros(shape=(2642, 2))
    test_l = labels[23765:]
    for i in range(train_y.shape[0]):
        train_y[i, train_l[i]] = 1
    for i in range(valid_y.shape[0]):
        valid_y[i, valid_l[i]] = 1
    for i in range(test_y.shape[0]):
        test_y[i, test_l[i]] = 1

    return train_x, train_y, valid_x, valid_y, test_x, test_y

In [3]:
# Parameters
import time
learning_rate = 0.0001
training_epochs = 100
batch_size = 100

# Construct model
pred, x, y = create_multilayer_perceptron()

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.compat.v1.global_variables_initializer()

# Load data
train_features, train_labels, valid_features, valid_labels, test_features, test_labels = preprocess()
t1 = time.time()
# Launch the graph in a session
with tf.compat.v1.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(train_features.shape[0] / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x = train_features[i * batch_size: (i + 1) * batch_size]
            batch_y = train_labels[i * batch_size: (i + 1) * batch_size]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch

        print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))

    print("Optimization Finished!")
    t2 = time.time()

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Time taken",str(t2-t1))
    print("Test Accuracy:", sess.run(accuracy, feed_dict={x: test_features, y: test_labels}))



2024-11-08 14:19:35.934635: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


Epoch: 0001 cost=385.402376943
Epoch: 0002 cost=214.690121691
Epoch: 0003 cost=166.760027989
Epoch: 0004 cost=142.171681517
Epoch: 0005 cost=124.903580779
Epoch: 0006 cost=111.217676316
Epoch: 0007 cost=100.280304850
Epoch: 0008 cost=91.959092398
Epoch: 0009 cost=85.181014065
Epoch: 0010 cost=79.480653582
Epoch: 0011 cost=74.409282684
Epoch: 0012 cost=69.771459516
Epoch: 0013 cost=65.811868383
Epoch: 0014 cost=62.481902538
Epoch: 0015 cost=59.238290900
Epoch: 0016 cost=56.454364899
Epoch: 0017 cost=54.000178753
Epoch: 0018 cost=51.782546898
Epoch: 0019 cost=49.898298155
Epoch: 0020 cost=48.151659012
Epoch: 0021 cost=46.339310533
Epoch: 0022 cost=44.660418072
Epoch: 0023 cost=43.249297196
Epoch: 0024 cost=41.919444482
Epoch: 0025 cost=40.619882412
Epoch: 0026 cost=39.180072102
Epoch: 0027 cost=37.981346017
Epoch: 0028 cost=36.922065057
Epoch: 0029 cost=35.831179678
Epoch: 0030 cost=34.790018644
Epoch: 0031 cost=33.872968764
Epoch: 0032 cost=32.944117720
Epoch: 0033 cost=32.016528489
Epo

In [5]:
# Parameters
import time
learning_rate = 0.0001
training_epochs = 4000
batch_size = 100

# Construct model
pred, x, y = create_multilayer_perceptron()

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.compat.v1.global_variables_initializer()

# Load data
train_features, train_labels, valid_features, valid_labels, test_features, test_labels = preprocess()
t1 = time.time()
# Launch the graph in a session
with tf.compat.v1.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(train_features.shape[0] / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x = train_features[i * batch_size: (i + 1) * batch_size]
            batch_y = train_labels[i * batch_size: (i + 1) * batch_size]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch

        print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))

    print("Optimization Finished!")
    t2 = time.time()

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Time taken",str(t2-t1))
    print("Test Accuracy:", sess.run(accuracy, feed_dict={x: test_features, y: test_labels}))



Epoch: 0001 cost=318.866482486
Epoch: 0002 cost=188.116781406
Epoch: 0003 cost=148.605249450
Epoch: 0004 cost=127.578758710
Epoch: 0005 cost=112.445332062
Epoch: 0006 cost=100.888532069
Epoch: 0007 cost=91.709338943
Epoch: 0008 cost=84.252400855
Epoch: 0009 cost=78.298472129
Epoch: 0010 cost=73.293102463
Epoch: 0011 cost=69.072747850
Epoch: 0012 cost=65.512444175
Epoch: 0013 cost=62.273046584
Epoch: 0014 cost=59.363958892
Epoch: 0015 cost=56.662741376
Epoch: 0016 cost=54.238157290
Epoch: 0017 cost=52.021853072
Epoch: 0018 cost=50.239929764
Epoch: 0019 cost=48.499058439
Epoch: 0020 cost=46.798340951
Epoch: 0021 cost=45.199410000
Epoch: 0022 cost=43.791598302
Epoch: 0023 cost=42.339281543
Epoch: 0024 cost=40.838116022
Epoch: 0025 cost=39.648299344
Epoch: 0026 cost=38.369073520
Epoch: 0027 cost=37.248833489
Epoch: 0028 cost=36.078780373
Epoch: 0029 cost=34.968276679
Epoch: 0030 cost=33.854955172
Epoch: 0031 cost=32.896042449
Epoch: 0032 cost=31.957732210
Epoch: 0033 cost=31.152888870
Epoc